# Prologue
Tensorflow（下稱TF）比剛推出的時候好上手很多了，其中一個主要的原因是其官方 wrapper: SLIM。
但在開始之前，我還是得強調一下 TF 和其他一般的數值運算編程(諸如 numpy、MATLAB等)的差異：

1. 區分為「符號(symbolic)」和「數值(numeric)」兩個部分。這是以前其他編程沒有的。
2. 必須先把 symoblic programming 的部分完成，也就是必須先將 neural network 的架構寫完。
3. 必須用 `tf.Session()`物件來聯繫 symbolic 和 numeric 兩個世界。


## 官方教學
話先講在前面：
1. 我覺得從官方教學下手是不錯，但比較難學。所以我才寫這份教學。
2. 我用的是 TF 的 Python API。TF 另有 C++ API，但我不曉得哪個研究人員會用。
3. 承上，如果沒用過 Python 的，請先自己用用看。Python 不會比 MATLAB 難。但視覺化是 MATLAB 的強項。

Tensorflow 官方提供了兩個版本的教學  
[官網版教學](https://www.tensorflow.org/versions/master/tutorials/index.html)  
這個教學講的是 Tensorflow 的用法。  
此教學分成有很多有趣的實作，  
從最簡單的 MNIST (數字辨識)、CIFAR10 (物件辨識)，乃至word2vec 以及機器翻譯都有。  
有興趣的話也可以去看他的Deep Dream。(上述全部都有提供code)  
其實如果真的學不起來也沒關係。直接修改他的 code 讓你的 task 可以跑就好了。  

[Udacity短課程](https://classroom.udacity.com/courses/ud730/)  
分四個小節講解：基本概念、DNN、CNN、RNN  
本課程的精隨在於他的作業  
作業是修改課程提供的範例code，如果熟 Tensorflow的話，改起來不算太難。  

所有的教學都一定要配~~溫開水~~[API](https://www.tensorflow.org/versions/master/api_docs/python/index.html) 服用

關於 Tensorflow 的安裝，請參考其[Github repository](https://github.com/tensorflow/tensorflow)  


# Symbolic Variables & Computational Graph
以前在寫其他的程式的時候，變數永遠是和值連繫在一起的 (e.g. `x = `)。  
但是在 TF 中，因為電腦要自動幫你算微分，而這只有 symbolic variable 才作得到。  
基本上，所有從 TF 呼叫或宣告得來的變數都是 symbolic。  

```python
w = tf.Variable(tf.zeros([128, 10]))
x = tf.zeros([10, 32])
y = tf.matmul(w, x)
```

你就可以要求 TF 計算 dy/dw 之類的東西。



Tensorflow 其實是在建立一張 computational graph of tensors，讓 data 可以 flow
以使 model 可以最終被 data 給塑型（故名 Tensorflow）。

Computational graph 是 Tensorflow 提供的 symbolic variables 的數學模組。  
讓你可以不用親手去導 gradient、不用 assign update rule。  

在建立 graph 的階段，computational graph 中的 symbolic operations 不會*立即*產生結果。  
相反的，它只是把「變數」和「運算」連結起來而已。  
一張典型的 computational graph 如下圖所示：  
![img](imgs/compugraph1.png)
本圖中：x (最底下的)經過一次 hidden layer，然後再經過 output layer，最後和 supervised target (y) 計算了 objective。  
  

放大其中一個模組：
![img](imgs/compugraph2.png)
在hidden layer 內，x 和 w 進行矩陣乘法，然後和 bias term 相加。最後再經過 ReLU 出去。  


我再用一個例子來講 symbolic 和 numeric 的差異。
舉例來說，在 MATLAB中
```octave
% Matlab
x = ones(128, 5)
y = x + 1.0
```  
最後一行的是 numerical operation，它會讓你立即得到 128x5 個 2.0

然而在 Tensorflow 中
```python
# Tensorflow
x = tf.ones(128, 5)
y = x + 1.0   # y = tf.add(x, 1.0)
```
最後一行其實只是 computational graph 的 construction/definition
你只會得到一個 symbolic variable，名叫 y。
它還不會真正有值。
除非你**啟動** graph (也就是呼叫 Session 物件，後面會講)。

## Remark
我用一個比喻來形容 computational graph：  
graph 中你所設計的 model parameters 就像是一塊大理石 (一開始它很奇形怪狀)，  
而 data 就像是瀑布的水或米開朗基羅的雕刻刀。  
Cost function 就像是你對作品的期許。  
作為雕刻刀的 data 不斷地銷過作為大理石的 parameters 的表面，最後你就會得到你的作品。